In [ ]:
# Parameters
day_obs = "2025-04-28"
day_obs_end = "2025-05-02"
instruments = "lsstcam, latiss, lsstcomcam"
variables = "airmass"

In [ ]:
day_obs_int = int(day_obs.replace("-", ""))
day_obs_end_int = int(day_obs_end.replace("-", ""))
instrument_list = [ins.strip() for ins in instruments.split(",")]

variables = variables.split(",") if len(variables) > 1 else variables
variables = [var.strip() for var in variables]

In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from IPython.display import display_markdown
import requests
import numpy as np
from datetime import datetime

In [ ]:
### 'Get Consdb access'
from lsst.summit.utils import ConsDbClient

URL = "http://consdb-pq.consdb:8080/consdb"  # Don't use this one
os.environ["no_proxy"] += ",.consdb"

client = ConsDbClient(URL)
display_markdown("### Consolidated Database is accessible", raw=True)

In [ ]:
%matplotlib inline
# After all imports

In [ ]:
# Put Plot in backend
plt.style.use("seaborn-v0_8-bright")


def plot(y, x, xname, yname, title=None):
    # plot
    fig = plt.figure(figsize=(20, 6), dpi=90)
    ax = fig.subplots()
    ax.scatter(x, y)

    # Get ticks at regular intervals
    step = 15
    tick_indices = np.arange(0, len(x), step)
    formatted_labels = [
        datetime.fromisoformat(x.iloc[i][:19].replace("T", " ")).strftime(
            "%Y-%m-%d %H:%M:%S"
        )
        for i in tick_indices
    ]

    ax.set_xticks(tick_indices)
    ax.set_xticklabels(formatted_labels, rotation=45)

    plt.xticks(rotation=20)
    plt.title(title)
    plt.xlabel(xname)
    plt.ylabel(yname)
    plt.tight_layout()  # ensures labels don’t get cut off
    plt.show()


def send_query(query_string, cdbclient):
    message = None
    data = None

    # print(query_string)
    try:
        data = cdbclient.query(query_string)
    except requests.HTTPError or requests.JSONDecodeError:
        data = cdbclient.query(query_string)
        message = "tried twice"
    except KeyError:
        message = "Attribute not in table"
    except ValueError:
        message = f"no data available for {query_string}"

    if message is not None:
        display_markdown(message, raw=True)

    return data


def plot_sub_df(plotvar, time, ogdf, instrument=None, table=None):
    subdf = ogdf[ogdf[plotvar].notna()]
    if len(subdf) == 0:
        print(f"Plot variable {plotvar} is empty for {instrument}")
    dt = subdf[time]
    if len(subdf) > 0:
        plot(subdf[plotvar], dt, time, plotvar, f"{instrument}.{table}.{plotvar}")

In [ ]:
for instrument in instrument_list:
    print("------------------------------------------------------------")
    print()
    display_markdown(f"# Instrument: {instrument}", raw=True)

    visit1_quicklook = f"""\
        SELECT * FROM cdb_{instrument}.visit1_quicklook \
        where day_obs >= {day_obs_int} and day_obs <= {day_obs_end_int}
    """

    visit1 = f"""\
        SELECT * FROM cdb_{instrument}.visit1 \
        where day_obs >= {day_obs_int} and day_obs <= {day_obs_end_int}
    """

    exposure = f"""\
        SELECT * FROM cdb_{instrument}.exposure \
        where day_obs >= {day_obs_int} and day_obs <= {day_obs_end_int}
    """

    # Join Visit1 and ccdVisit1 to access data and day obs
    quicklook = send_query(visit1_quicklook, client)
    visits = send_query(visit1, client)

    quicklook = quicklook.to_pandas() if quicklook else None
    visits = visits.to_pandas() if visits else None

    """ 
    When Quicklook exists again, join it to exposures, 
    so we can access obs_start and plot the calculated variable against it.
    """
    if quicklook is not None:
        #        This join is happening incorrectly inserting nans
        visits = visits.join(quicklook, on="visit_id", lsuffix="", rsuffix="_q")

    vars_to_plot = [
        "dimm_seeing",
        "seeing_zenith_500nm_median",
        "psf_trace_radius_delta_median",
        "airmass",
        "zero_point",
        "zero_point_median",
        "sky_bg_median",
        "obs_start",
    ]

    for var in variables:
        if var not in vars_to_plot:
            vars_to_plot.append(var)

    display_markdown(f"Plotting variables: {vars_to_plot}", raw=True)
    if visits is not None:
        for variable in vars_to_plot:
            try:
                plot_sub_df(variable, "obs_start", visits, instrument, "exposure")
            except KeyError:
                print(f"Skipping {variable}, not present in this table")
                continue

        # to print to a file visits.to_json(f"{instrument}{day_obs_int}.json

In [ ]:
instrument = "lsstcam"
visit1_quicklook = f"""\
    SELECT * FROM cdb_{instrument}.visit1_quicklook \
    where day_obs >= {day_obs_int} and day_obs <= {day_obs_end_int}
"""
visit1 = f"""\
    SELECT * FROM cdb_{instrument}.visit1 \
    where day_obs >= {day_obs_int} and day_obs <= {day_obs_end_int}
"""
print(visit1_quicklook)
print(visit1)
quickdata = send_query(visit1_quicklook, client).to_pandas()
visitdata = send_query(visit1, client).to_pandas()
qvid = quickdata["visit_id"]
vvid = visitdata["visit_id"]
if qvid == vvid:
    print("welp")

In [ ]:
print(quickdata["psf_trace_radius_delta_median"])
# print(visitdata)

newdata = visitdata.join(quickdata, on="visit_id", how="left", lsuffix="", rsuffix="_q")
newdata = pd.merge(
    quickdata,
    visitdata,
)
varsx = [
    "dimm_seeing",
    "seeing_zenith_500nm_median",
    "psf_trace_radius_delta_median",
    "airmass",
    "zero_point_median",
    "sky_bg_median",
    "obs_start",
]
for var in varsx:
    print(newdata[var])